# Bayesian Writing Sample Factor Data Processing

## Methodology

In [ ]:
import pandas as pd

# Data source: WRDS
from wrds import Connection

# Data output
import csv

In [ ]:
def convert_to_weekly(df):
    """
    Convert daily financial data to weekly, starting from 2010/01/03
    
    Parameters:
    df (pandas.DataFrame): DataFrame with date index and financial metrics
    
    Returns:
    pandas.DataFrame: Weekly resampled data
    """
    # Convert the 'date' column to datetime if it's not already
    df['date'] = pd.to_datetime(df['date'])
    
    # Set date as index
    df = df.set_index('date')
    
    # Resample to weekly frequency (week ending on Sunday)
    # Using last() to get the last value of each week
    weekly_data = df.resample('W-SUN').last()
    
    # Reset index to make date a column again
    weekly_data = weekly_data.reset_index()
    
    return weekly_data


In [4]:
def slice_data(df, start_date='2010/01/03', end_date='2019/12/29'):
    """
    Slice the data between specified start and end dates
    
    Parameters:
    df (pandas.DataFrame): DataFrame with 'date' column
    start_date (str): Start date in YYYY/MM/DD format
    end_date (str): End date in YYYY/MM/DD format
    
    Returns:
    pandas.DataFrame: Sliced data
    """
    # Create a copy of the dataframe
    df = df.copy()
    
    # Convert dates to datetime for comparison
    df['date'] = pd.to_datetime(df['date'])
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Slice the data
    mask = (df['date'] >= start_date) & (df['date'] <= end_date)
    sliced_data = df.loc[mask].copy()  # Create explicit copy of the slice
    
    # Convert dates back to string format
    sliced_data['date'] = sliced_data['date'].dt.strftime('%Y/%m/%d')
    
    return sliced_data

## Fama French 5 Factors Plus Momentum

Data source: WRDS

In [5]:
FF5M = pd.read_csv('data\FamaFrenchPlusMomentum.csv')
FF5M = convert_to_weekly(FF5M)
FF5M = slice_data(FF5M)

print(FF5M)

           date   mktrf     smb     hml     rmw     cma       rf     umd
4    2010/01/03 -0.0099 -0.0017  0.0039 -0.0047  0.0017  0.00000  0.0003
5    2010/01/10  0.0033  0.0032  0.0001  0.0022 -0.0037  0.00000  0.0020
6    2010/01/17 -0.0112 -0.0019 -0.0039  0.0061 -0.0015  0.00000 -0.0052
7    2010/01/24 -0.0214  0.0051 -0.0089  0.0046  0.0000  0.00000 -0.0158
8    2010/01/31 -0.0097 -0.0008 -0.0042 -0.0020  0.0018  0.00000 -0.0124
..          ...     ...     ...     ...     ...     ...      ...     ...
521  2019/12/01 -0.0042 -0.0011 -0.0031 -0.0047  0.0000  0.00006 -0.0001
522  2019/12/08  0.0091  0.0038  0.0037  0.0033  0.0022  0.00007 -0.0076
523  2019/12/15 -0.0003 -0.0045 -0.0055 -0.0027 -0.0024  0.00007  0.0076
524  2019/12/22  0.0048 -0.0027 -0.0031  0.0001 -0.0007  0.00007  0.0023
525  2019/12/29 -0.0010 -0.0055 -0.0007  0.0026  0.0010  0.00007  0.0037

[522 rows x 8 columns]


## Q-factors

Source: WRDS

In [1]:
# Connect to WRDS
conn = Connection()

# Run your query
Q_Factors = conn.raw_sql("SELECT * FROM macrofin.q_factors_daily")

# Save to a CSV file if desired
# Close the connection
conn.close()

NameError: name 'Connection' is not defined

In [7]:
Q_Factors = convert_to_weekly(Q_Factors)
Q_Factors = slice_data(Q_Factors)
print(Q_Factors)

            date     r_f   r_mkt    r_me    r_ia   r_roe    r_eg
2243  2010/01/03  0.0003 -0.8664 -0.0883  0.1901 -0.0024 -0.0776
2244  2010/01/10  0.0001  0.4160  0.2017 -0.4278 -0.1254  0.0851
2245  2010/01/17  0.0001 -1.1324 -0.2926 -0.0268  0.4190  0.2474
2246  2010/01/24  0.0001 -2.0990  0.2744  0.0938  0.1175 -0.4539
2247  2010/01/31  0.0001 -1.0283  0.0259  0.3848  0.2150 -0.2583
...          ...     ...     ...     ...     ...     ...     ...
2760  2019/12/01  0.0060 -0.4105 -0.1160 -0.0309 -0.1402 -0.0431
2761  2019/12/08  0.0068  0.8357  0.3910  0.1494 -0.2265 -0.1986
2762  2019/12/15  0.0068 -0.0037 -0.5008 -0.3819  0.0556  0.3914
2763  2019/12/22  0.0068  0.4472 -0.4079  0.0509  0.0721  0.1331
2764  2019/12/29  0.0068 -0.0711 -0.5205  0.0495  0.3771  0.2624

[522 rows x 7 columns]


## Combination and Output

In [8]:
# Drop the 'date' column from Q_Factors
Q_Factors = Q_Factors.drop(columns=['date'])

# Ensure both DataFrames have the same number of rows
if len(Q_Factors) != len(FF5M):
    print("Warning: The two DataFrames have different lengths!")
    # Adjust to the minimum length
    min_len = min(len(Q_Factors), len(FF5M))
    Q_Factors = Q_Factors.iloc[:min_len]
    FF5M = FF5M.iloc[:min_len]

# Concatenate the DataFrames side by side (excluding the date column from Q_Factors)
combined_df = pd.concat([FF5M.reset_index(drop=True), Q_Factors.reset_index(drop=True)], axis=1)

# Display the combined DataFrame
print(combined_df)

           date   mktrf     smb     hml     rmw     cma       rf     umd  \
0    2010/01/03 -0.0099 -0.0017  0.0039 -0.0047  0.0017  0.00000  0.0003   
1    2010/01/10  0.0033  0.0032  0.0001  0.0022 -0.0037  0.00000  0.0020   
2    2010/01/17 -0.0112 -0.0019 -0.0039  0.0061 -0.0015  0.00000 -0.0052   
3    2010/01/24 -0.0214  0.0051 -0.0089  0.0046  0.0000  0.00000 -0.0158   
4    2010/01/31 -0.0097 -0.0008 -0.0042 -0.0020  0.0018  0.00000 -0.0124   
..          ...     ...     ...     ...     ...     ...      ...     ...   
517  2019/12/01 -0.0042 -0.0011 -0.0031 -0.0047  0.0000  0.00006 -0.0001   
518  2019/12/08  0.0091  0.0038  0.0037  0.0033  0.0022  0.00007 -0.0076   
519  2019/12/15 -0.0003 -0.0045 -0.0055 -0.0027 -0.0024  0.00007  0.0076   
520  2019/12/22  0.0048 -0.0027 -0.0031  0.0001 -0.0007  0.00007  0.0023   
521  2019/12/29 -0.0010 -0.0055 -0.0007  0.0026  0.0010  0.00007  0.0037   

        r_f   r_mkt    r_me    r_ia   r_roe    r_eg  
0    0.0003 -0.8664 -0.0883  0.19

In [9]:
# Method 1: Using csv writer
with open('data\Factors.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # Write headers
    writer.writerow(combined_df.columns)
    # Write data
    writer.writerows(combined_df.values)